In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from Utilities.get_image_data import load_data
from Utilities.form_train_val_test_batches import form_train_val_test_batches
from Utilities.NN_FC_layerwise import FCLayerwise
from Utilities.NN_FC_layerwise_new import FCLayerwise_new
from Utilities.Net import Final_Network
from Utilities.Net_new import Final_Network_ALGO_II
from Utilities.create_data import create_new
from Utilities.loss_and_accuracies import data_loss_classification, accuracy_classification
from Utilities.manifold_regularization import manifold_classification
from Utilities.manifold_regularization_new import manifold_classification_new
from Utilities.optimize_layerwise import optimize
from Utilities.additive_output import net_output 
from Utilities.EXTRA_ACCURATE_NETWORK_OPTIMIZE import optimize_new
from Utilities.EXTRA_ACCURATE_NETWORK import FCLayerwise_extra_accurate
from Utilities.plot_and_save_figures_layerwise import plot_fig
from joblib import Parallel, delayed
import multiprocessing
import random

In [2]:
from decimal import Decimal # for filenames

import pdb #Equivalent of keyboard in MATLAB, just add "pdb.set_trace()"

import os
import sys

In [3]:
###############################################################################
#                       HyperParameters and RunOptions                        #
###############################################################################
class Hyperparameters:
    max_hidden_layers = 9 # For this architecture, need at least 2. One for the mapping to the feature space, one as a trainable hidden layer. EXCLUDES MAPPING BACK TO DATA SPACE
    filter_size       = 3 
    num_filters       = 64
    activation        ='elu'
    classification_act= 'linear'
    regularization    = 0.001
    manifold          = 0.01
    node_TOL          = 1e-4
    error_TOL         = 1e-4
    batch_size        = 1000
    num_epochs        = 40
    
    num_networks      = 6
    

In [4]:
class Hyperparameters_new:
    max_hidden_layers = 3 # For this architecture, need at least 2. One for the mapping to the feature space, one as a trainable hidden layer. EXCLUDES MAPPING BACK TO DATA SPACE
    num_hidden_nodes  = 100
    activation        = 'elu'
    classification_act= 'linear'
    regularization    = 0.000
    manifold          = 0.000
    node_TOL          = 1e-4
    error_TOL         = 1e-4
    batch_size        = 1000
    num_epochs        = 30

In [5]:
class RunOptions:
    def __init__(self):    
        #=== Choose Which GPU to Use ===#
        self.which_gpu = '1'
        
        #=== Use L_1 Regularization ===#
        self.use_L1 = 1
        
        #=== Choose Data Set ===#
        self.data_MNIST = 0
        self.data_CIFAR10 = 1 
        self.data_CIFAR100 = 0
        
        #=== Random Seed ===#
        self.random_seed = 1234


In [6]:
###############################################################################
#                                 File Paths                                  #
###############################################################################         
class FilePaths():    
    def __init__(self, hyperp, run_options):  
        #=== Declaring File Name Components ===# 
        self.NN_type = 'CNN'
        if run_options.data_MNIST == 1:
            self.dataset = 'MNIST'
        if run_options.data_CIFAR10 == 1:
            self.dataset = 'CIFAR10'
        if run_options.data_CIFAR100 == 1:
            self.dataset = 'CIFAR100'
        if hyperp.regularization >= 1:
            hyperp.regularization = int(hyperp.regularization)
            regularization_string = str(hyperp.regularization)
        else:
            regularization_string = str(hyperp.regularization)
            regularization_string = 'pt' + regularization_string[2:]                        
        node_TOL_string = str('%.2e' %Decimal(hyperp.node_TOL))
        node_TOL_string = node_TOL_string[-1]
        error_TOL_string = str('%.2e' %Decimal(hyperp.error_TOL))
        error_TOL_string = error_TOL_string[-1]
        
        #=== File Name ===#
        if run_options.use_L1 == 0:
            self.filename = self.dataset + '_' + self.NN_type + '_mhl%d_hl%d_eTOL%s_b%d_e%d' %(hyperp.max_hidden_layers, hyperp.num_filters, error_TOL_string, hyperp.batch_size, hyperp.num_epochs)
        else:
            self.filename = self.dataset + '_' + self.NN_type + '_L1_mhl%d_hl%d_r%s_nTOL%s_eTOL%s_b%d_e%d' %(hyperp.max_hidden_layers, hyperp.num_filters, regularization_string, node_TOL_string, error_TOL_string, hyperp.batch_size, hyperp.num_epochs)

        #=== Saving Trained Neural Network and Tensorboard ===#
        #self.NN_savefile_directory = 'C:/Users/Chandradut/Desktop/Sparse training/Trained_NNs/' + self.filename # Since we need to save four different types of files to save a neural network model, we need to create a new folder for each model
        self.NN_savefile_directory =  self.filename
        self.NN_savefile_name = self.NN_savefile_directory + '/' + self.filename # The file path and name for the four files
        #self.tensorboard_directory = 'C:/Users/Chandradut/Desktop/Sparse training/Tensorboard/' + self.filename

###############################################################################



In [7]:
    #=== Hyperparameters and Run Options ===#    
    hyperp = Hyperparameters()
    hyperp_new=Hyperparameters_new()
    run_options = RunOptions()
    

    #=== File Names ===#
    file_paths = FilePaths(hyperp, run_options)
    
   # === Load Data ===#       
    data_train, labels_train,\
    data_test, labels_test,\
    data_input_shape, num_channels, label_dimensions\
    = load_data(file_paths.NN_type, file_paths.dataset, run_options.random_seed) 
            
            #=== GPU Settings ===#
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = run_options.which_gpu
    
            #=== Neural Network ===#
    if run_options.use_L1 == 0:
        kernel_regularizer = None
        bias_regularizer = None  
    else:
        kernel_regularizer = tf.keras.regularizers.l1(hyperp.regularization)
        bias_regularizer = tf.keras.regularizers.l1(hyperp.regularization)

    y_true = tf.one_hot(tf.cast(labels_train,tf.int64), label_dimensions, dtype=tf.float32)
    new_label=np.squeeze(y_true,axis=1)
    
    data_and_labels_train, data_and_labels_val, data_and_labels_test,\
    num_data_train, num_data_val, num_data_test,\
    num_batches_train, num_batches_val, num_batches_test,data_and_labels_train_new\
    = form_train_val_test_batches(data_train, labels_train, \
                                data_test, labels_test, \
                                hyperp.batch_size, new_label, run_options.random_seed)
        
  



trainable_hidden_layer_index=8
NN=FCLayerwise_extra_accurate( hyperp,run_options, data_input_shape, label_dimensions,trainable_hidden_layer_index) 
        
#NN.load_weights("WEIGHTS_AFTER_ALGORITHM_I"+'/'+"model_weights"+str(1)).expect_partial()
hyperp_n=hyperp
optimize_new(hyperp,hyperp_n, run_options, file_paths, NN, data_loss_classification, accuracy_classification, data_and_labels_train, data_and_labels_val, data_and_labels_test, label_dimensions, num_batches_train,data_and_labels_train_new,hyperp.batch_size,run_options.random_seed,num_data_train,1,data_input_shape)   
           #=== Optimizer ===#

        
        
        
        
        
        
        
        

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
ListWrapper([ListWrapper([32, 3]), ListWrapper([3, 64]), ListWrapper([3, 64]), 10])
Beginning Training
            Epoch 0            
CIFAR10_CNN_L1_mhl9_hl64_rpt001_nTOL4_eTOL4_b1000_e40
Trainable Hidden Layer Index: 8
GPU: 1

Optimizing 40 batches of size 1000:
Model: "fc_layerwise_extra_accurate"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
W2 (Conv2D)                  multiple                  36928     
_________________________________________________________________
W3 (Conv2D)                  multiple                  36928     
_________________________________________________________________
W4 (Conv2D)                  multiple                  36928     
_________________________________________________________________
W5 (Conv2D)                  multiple                  36928     
__________________________

Time per Batch: 2.56
Time per Epoch: 114.54

Training Set: Loss: 5.919e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 0.000
Test Set: Loss: 0.000e+00, Accuracy: 0.707

Previous Layer Relative # of 0s: 0.0000000

            Epoch 14            
CIFAR10_CNN_L1_mhl9_hl64_rpt001_nTOL4_eTOL4_b1000_e40
Trainable Hidden Layer Index: 8
GPU: 1

Optimizing 40 batches of size 1000:
Time per Batch: 2.88
Time per Epoch: 115.02

Training Set: Loss: 5.696e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 0.000
Test Set: Loss: 0.000e+00, Accuracy: 0.719

Previous Layer Relative # of 0s: 0.0000000

            Epoch 15            
CIFAR10_CNN_L1_mhl9_hl64_rpt001_nTOL4_eTOL4_b1000_e40
Trainable Hidden Layer Index: 8
GPU: 1

Optimizing 40 batches of size 1000:
Time per Batch: 2.83
Time per Epoch: 114.90

Training Set: Loss: 5.344e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 0.000
Test Set: Loss: 0.000e+00, Accuracy: 0.716

Previous Layer Relative # of 0s

Time per Batch: 2.55
Time per Epoch: 115.69

Training Set: Loss: 1.319e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 0.000
Test Set: Loss: 0.000e+00, Accuracy: 0.729

Previous Layer Relative # of 0s: 0.0000000

            Epoch 32            
CIFAR10_CNN_L1_mhl9_hl64_rpt001_nTOL4_eTOL4_b1000_e40
Trainable Hidden Layer Index: 8
GPU: 1

Optimizing 40 batches of size 1000:
Time per Batch: 2.59
Time per Epoch: 115.83

Training Set: Loss: 1.278e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 0.000
Test Set: Loss: 0.000e+00, Accuracy: 0.725

Previous Layer Relative # of 0s: 0.0000000

            Epoch 33            
CIFAR10_CNN_L1_mhl9_hl64_rpt001_nTOL4_eTOL4_b1000_e40
Trainable Hidden Layer Index: 8
GPU: 1

Optimizing 40 batches of size 1000:
Time per Batch: 2.52
Time per Epoch: 117.91

Training Set: Loss: 1.053e-01, Accuracy: 0.000
Validation Set: Loss: 0.000e+00, Accuracy: 0.000
Test Set: Loss: 0.000e+00, Accuracy: 0.728

Previous Layer Relative # of 0s

In [9]:
plot_fig(hyperp, run_options, file_paths,1)

FileNotFoundError: [Errno 2] No such file or directory: 'CIFAR10_CNN_L1_mhl9_hl20_rpt001_nTOL4_eTOL4_b1000_e10/CIFAR10_CNN_L1_mhl9_hl20_rpt001_nTOL4_eTOL4_b1000_e10_metrics_hl21.csv'

In [7]:
    #=== Hyperparameters and Run Options ===#    
    hyperp = Hyperparameters()
    hyperp_new=Hyperparameters_new()
    run_options = RunOptions()
    

    #=== File Names ===#
    file_paths = FilePaths(hyperp, run_options)
    
   # === Load Data ===#       
    data_train, labels_train,\
    data_test, labels_test,\
    data_input_shape, num_channels, label_dimensions\
    = load_data(file_paths.NN_type, file_paths.dataset, run_options.random_seed) 
    

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [8]:
NN.save_weights("WEIGHTS"+'/'+"model_weights"+str(1))

In [9]:
        
            #=== GPU Settings ===#
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = run_options.which_gpu
    
            #=== Neural Network ===#
if run_options.use_L1 == 0:
    kernel_regularizer = None
    bias_regularizer = None  
else:
    kernel_regularizer = tf.keras.regularizers.l1(hyperp.regularization)
    bias_regularizer = tf.keras.regularizers.l1(hyperp.regularization)


In [12]:
        data_and_labels_train, data_and_labels_val, data_and_labels_test,\
        num_data_train, num_data_val, num_data_test,\
        num_batches_train, num_batches_val, num_batches_test,data_and_labels_train_new\
        = form_train_val_test_batches(data_train, labels_train, \
                                  data_test, labels_test, \
                                  hyperp.batch_size, new_label, run_options.random_seed)
        

In [11]:
data_train,new_label,labels_train=create_new(0,data_train, labels_train,hyperp,hyperp_new, run_options, data_input_shape, label_dimensions,1)
       

In [17]:
trainable_hidden_layer_index=8
NN=FCLayerwise_extra_accurate( hyperp,run_options, data_input_shape, label_dimensions,trainable_hidden_layer_index) 
        
NN.load_weights("WEIGHTS"+'/'+"model_weights"+str(1)).expect_partial()
hyperp_n=hyperp
optimize_new(hyperp,hyperp_n, run_options, file_paths, NN, data_loss_classification, accuracy_classification, data_and_labels_train, data_and_labels_val, data_and_labels_test, label_dimensions, num_batches_train,data_and_labels_train_new,hyperp.batch_size,run_options.random_seed,num_data_train,1,data_input_shape)   
           #=== Optimizer ===#

ListWrapper([ListWrapper([32, 3]), ListWrapper([3, 20]), ListWrapper([3, 20]), 10])
Beginning Training
            Epoch 0            
CIFAR10_CNN_L1_mhl9_hl20_rpt001_nTOL4_eTOL4_b1000_e1
Trainable Hidden Layer Index: 8
GPU: 1

Optimizing 40 batches of size 1000:


ValueError: too many values to unpack (expected 2)

In [8]:
NN.save_weights("WEIGHTS"+'/'+"model_weights"+str(2))

In [9]:
    file_paths = FilePaths(hyperp, run_options)
    
    #=== Load Data ===#       
    data_train, labels_train,\
    data_test, labels_test,\
    data_input_shape, num_channels, label_dimensions\
    = load_data(file_paths.NN_type, file_paths.dataset, run_options.random_seed) 
        

In [10]:
            Network=Final_Network( hyperp,run_options, data_input_shape, label_dimensions) 
        
            Network.load_weights("WEIGHTS"+'/'+"model_weights"+str(2-1)).expect_partial()

ListWrapper([ListWrapper([32, 3]), ListWrapper([3, 20]), ListWrapper([3, 20]), 10])


In [13]:
bb=Network(batch_data_test)

In [25]:
for batch_data_test, batch_labels_test in data_and_labels_test:
    b=batch_data_test

In [24]:
data = tf.reshape(batch_data_test, (len(batch_data_test), 32*32*3))
batch_pred_test,val = NN(data)

In [15]:
batch_pred_test_new=batch_pred_test

In [23]:
rnew=bb+batch_pred_test_new

In [26]:
accuracy_classification(rnew, batch_labels_test)

<tf.Tensor: shape=(1000,), dtype=float32, numpy=
array([1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       1., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0.,
       0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1.,
       0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1.,
       0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1.,
       0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1.,
       1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 

In [99]:
y_pred=Network(data_train[0:1000])
for i in range(1000,50000,1000):
    
    r=Network(data_train[i:i+1000])
    

    y_pred=tf.concat([y_pred,r],0)

In [97]:
y_pred=Network(data_train[0:1000])

In [10]:
y_true = tf.one_hot(tf.cast(labels_train,tf.int64), label_dimensions, dtype=tf.float32)


In [11]:
y_true

<tf.Tensor: shape=(50000, 1, 10), dtype=float32, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 1.]],

       [[0., 0., 0., ..., 0., 0., 1.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 1.]],

       [[0., 1., 0., ..., 0., 0., 0.]],

       [[0., 1., 0., ..., 0., 0., 0.]]], dtype=float32)>

In [12]:
y_true=np.squeeze(y_true,axis=1)
y_true

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [15]:
np.expand_dims(y_true,axis=1)

array([[[0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 1.]],

       [[0., 0., 0., ..., 0., 0., 1.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 1.]],

       [[0., 1., 0., ..., 0., 0., 0.]],

       [[0., 1., 0., ..., 0., 0., 0.]]], dtype=float32)

In [17]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import datasets

(data_train, labels_train), (data_test, labels_test) = datasets.cifar10.load_data()
#(data_train, labels_train), (data_test, labels_test) = datasets.mnist.load_data()
#data_train = tf.reshape(data_train, (len(data_train), 28*28))
#data_test = tf.reshape(data_test, (len(data_test), 28*28))
#data_train = tf.reshape(data_train, (len(data_train), 32*32*3))
#data_test = tf.reshape(data_test, (len(data_test), 32*32*3))
    
label_dimensions = 10
data_input_shape = data_train.shape[1:]

In [19]:
len(data_train)

50000

In [20]:
data_train.shape[-1]

3

In [14]:
data_train

<tf.Tensor: shape=(50000, 3072), dtype=uint8, numpy=
array([[ 59,  62,  63, ..., 123,  92,  72],
       [154, 177, 187, ..., 143, 133, 144],
       [255, 255, 255, ...,  80,  86,  84],
       ...,
       [ 35, 178, 235, ...,  12,  31,  50],
       [189, 211, 240, ..., 195, 190, 171],
       [229, 229, 239, ..., 163, 163, 161]], dtype=uint8)>

In [7]:
      #=== Hyperparameters and Run Options ===#    
hyperp = Hyperparameters()
hyperp_new=Hyperparameters_new()
run_options = RunOptions()
    

    #=== File Names ===#
file_paths = FilePaths(hyperp, run_options)
    #=== Load Data ===#       
data_train, labels_train,data_test, labels_test,data_input_shape, num_channels, label_dimensions= load_data(file_paths.NN_type, file_paths.dataset, run_options.random_seed) 
    

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [10]:
data_train

array([[[[ 59,  62,  63],
         [ 43,  46,  45],
         [ 50,  48,  43],
         ...,
         [158, 132, 108],
         [152, 125, 102],
         [148, 124, 103]],

        [[ 16,  20,  20],
         [  0,   0,   0],
         [ 18,   8,   0],
         ...,
         [123,  88,  55],
         [119,  83,  50],
         [122,  87,  57]],

        [[ 25,  24,  21],
         [ 16,   7,   0],
         [ 49,  27,   8],
         ...,
         [118,  84,  50],
         [120,  84,  50],
         [109,  73,  42]],

        ...,

        [[208, 170,  96],
         [201, 153,  34],
         [198, 161,  26],
         ...,
         [160, 133,  70],
         [ 56,  31,   7],
         [ 53,  34,  20]],

        [[180, 139,  96],
         [173, 123,  42],
         [186, 144,  30],
         ...,
         [184, 148,  94],
         [ 97,  62,  34],
         [ 83,  53,  34]],

        [[177, 144, 116],
         [168, 129,  94],
         [179, 142,  87],
         ...,
         [216, 184, 140],
        

In [9]:
x_train_new = data_train[labels_train == 0]

In [17]:
length=int((len(x_train_new))/2)
x_train_new[0:length]

<tf.Tensor: shape=(2500, 32, 32, 3), dtype=float32, numpy=
array([[[[ 0.5844234 ,  0.60631365,  0.551588  ],
         [ 0.5844234 ,  0.60631365,  0.551588  ],
         [ 0.60631365,  0.6282039 ,  0.5734783 ],
         ...,
         [ 0.63914907,  0.6500942 ,  0.56253314],
         [ 0.6172588 ,  0.6282039 ,  0.551588  ],
         [ 0.5953685 ,  0.60631365,  0.54064286]],

        [[ 0.6282039 ,  0.6500942 ,  0.5953685 ],
         [ 0.6282039 ,  0.6500942 ,  0.5953685 ],
         [ 0.63914907,  0.66103935,  0.60631365],
         ...,
         [ 0.6719845 ,  0.68292964,  0.6172588 ],
         [ 0.6500942 ,  0.66103935,  0.5953685 ],
         [ 0.6282039 ,  0.63914907,  0.5734783 ]],

        [[ 0.6719845 ,  0.6938748 ,  0.63914907],
         [ 0.68292964,  0.7048199 ,  0.6500942 ],
         [ 0.6938748 ,  0.71576506,  0.6719845 ],
         ...,
         [ 0.71576506,  0.71576506,  0.6719845 ],
         [ 0.6938748 ,  0.7048199 ,  0.6500942 ],
         [ 0.6719845 ,  0.68292964,  0.628203

In [15]:
from tensorflow.keras.layers import Dense, Conv2D, Flatten, BatchNormalization
vall=Flatten()(x_train_new)

<tf.Tensor: shape=(3072,), dtype=float32, numpy=
array([0.5844234 , 0.60631365, 0.551588  , ..., 1.0003387 , 0.97844833,
       1.033174  ], dtype=float32)>

In [22]:
architecture.append([data_input_shape[0],3]) # input information

In [24]:
architecture.append([3,10]) # Upsampling data

In [25]:
architecture[1][1]

10

In [ ]:
labels_train=np.squeeze(labels_train,axis=1)

In [ ]:
Network=Final_Network(hyperp, run_options, data_input_shape, label_dimensions,
                      kernel_regularizer, bias_regularizer) 

In [ ]:
Network.load_weights("WEIGHTS"+'/'+"model_weights1")

In [ ]:
(data_train, labels_train), (data_test, labels_test) = datasets.mnist.load_data()
data_train = data_train.reshape(data_train.shape[0], 28, 28, 1)
data_test = data_test.reshape(data_test.shape[0], 28, 28, 1)
label_dimensions = 10
data_input_shape = data_train.shape[1:]

#=== Casting as float32 ===#
data_train = tf.cast(data_train,tf.float32)
labels_train = tf.cast(labels_train, tf.int32)
data_test = tf.cast(data_test, tf.float32)
labels_test = tf.cast(labels_test, tf.int32)
    
#=== Normalize Data ===#
data_train, data_test = data_train/255.0, data_test/255.0
data_train = tf.image.per_image_standardization(data_train) # Linearly scales each image to have mean 0 and variance 1
data_test = tf.image.per_image_standardization(data_test)   # Linearly scales each image to have mean 0 and variance 1

data_train = tf.reshape(data_train, (len(data_train), 28*28))
data_test = tf.reshape(data_test, (len(data_test), 28*28))

num_data_train = len(data_train)
num_data_test = len(data_test)
random_seed=1234
batch_size        = 1000

data_and_labels_train_full = tf.data.Dataset.from_tensor_slices((data_train, labels_train)).shuffle(num_data_train, seed=random_seed)
data_and_labels_test = tf.data.Dataset.from_tensor_slices((data_test, labels_test)).batch(batch_size)
num_batches_test = len(list(data_and_labels_test))

#=== Partitioning Out Validation Set and Constructing Batches ===#
current_num_data_train = num_data_train
num_data_train = int(0.8 * num_data_train)
num_data_val = current_num_data_train - num_data_train
data_and_labels_train_full = tf.data.Dataset.from_tensor_slices((data_train, labels_train)).shuffle(num_data_train, seed=random_seed)
data_and_labels_train = data_and_labels_train_full.take(num_data_train).batch(batch_size)
data_and_labels_val = data_and_labels_train_full.skip(num_data_train).batch(batch_size)    
num_batches_train = len(list(data_and_labels_train))
num_batches_val = len(list(data_and_labels_val))

In [ ]:
(data_train, labels_train), (data_test, labels_test) = datasets.mnist.load_data()
data_train = data_train.reshape(data_train.shape[0], 28, 28, 1)
data_test = data_test.reshape(data_test.shape[0], 28, 28, 1)
label_dimensions = 10
data_input_shape = data_train.shape[1:]

#=== Casting as float32 ===#
data_train = tf.cast(data_train,tf.float32)
labels_train = tf.cast(labels_train, tf.int32)
data_test = tf.cast(data_test, tf.float32)
labels_test = tf.cast(labels_test, tf.int32)
    
#=== Normalize Data ===#
data_train, data_test = data_train/255.0, data_test/255.0
data_train = tf.image.per_image_standardization(data_train) # Linearly scales each image to have mean 0 and variance 1
data_test = tf.image.per_image_standardization(data_test)   # Linearly scales each image to have mean 0 and variance 1

data_train = tf.reshape(data_train, (len(data_train), 28*28))
data_test = tf.reshape(data_test, (len(data_test), 28*28))

num_data_train = len(data_train)
num_data_test = len(data_test)
random_seed=1234
batch_size        = 1000

In [ ]:
labels=tf.one_hot(tf.cast(labels_train,tf.int64), label_dimensions, dtype=tf.float32)

In [ ]:
data_and_labels_train_full = tf.data.Dataset.from_tensor_slices((data_train, labels_train,labels)).shuffle(num_data_train, seed=random_seed)

In [ ]:
data_and_labels_train = data_and_labels_train_full.take(num_data_train).batch(batch_size)
data_and_labels_val = data_and_labels_train_full.skip(num_data_train).batch(batch_size)    
num_batches_train = len(list(data_and_labels_train))
num_batches_val = len(list(data_and_labels_val))

In [ ]:
for batch_num, (batch_data_train, batch_labels_train,labels) in data_and_labels_train.enumerate():
    batch=batch_num
    batch_data_train = batch_data_train
    batch_labels_train=batch_labels_train
    lab=labels

In [ ]:
lab


In [ ]:
y_true = tf.one_hot(tf.cast(labels_train,tf.int64), label_dimensions, dtype=tf.float32)

In [ ]:
y_true

In [ ]:
np.mean(tf.keras.losses.mean_squared_error(new_one, val[0:dimension[0]-1]))

In [ ]:

x_train_new = batch_data_train[batch_labels_train == 1]
batch_pred_train,val=NN(x_train_new)



In [ ]:
y_true = tf.one_hot(tf.cast(y_true,tf.int64), label_dimensions, dtype=tf.float32)

In [ ]:
loss

In [ ]:
np.linalg.norm(val[0]-val[1])